# Exploring Positional Encoding Effects in Transformer Classifiers

This notebook investigates how different positional encoding strategies affect transformer representations for the 4t vs background classification task.

## Overview

We'll analyze four trained models with different positional encodings:
1. **None** - No positional information (permutation invariant)
2. **Sinusoidal** - Fixed sin/cos patterns from "Attention is All You Need"
3. **Learned** - Trainable positional embeddings
4. **Rotary (RoPE)** - Rotary position embeddings applied in attention

## Key Questions
- How do positional encodings look in model dimensions?
- How do they affect representations layer-by-layer through the transformer?
- Why does positional encoding improve performance?
- What's the difference between sinusoidal, learned, and rotary approaches?

In [ ]:
# Imports
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from omegaconf import OmegaConf

# Add project root to path
project_root = Path(r"C:\Users\niels\Projects\Thesis-Code\Code\Niels_repo")
sys.path.insert(0, str(project_root / "src"))  # noqa: E402

from thesis_ml.data.h5_loader import make_classification_dataloaders  # noqa: E402

# Plotting setup
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
%matplotlib inline

print("✓ Imports successful")

## 1. Load All Four Models

Load the trained models with different positional encodings from the experiment directory.

In [ ]:
# Define paths
models_root = Path(r"C:\Users\niels\Projects\Thesis-Code\Saved models\20251127-162910_4t_vs_background_postional")
data_path = Path(r"C:\Users\niels\Projects\Thesis-Code\Data\4tops_splitted.h5")

# Load all four models
models_info = {}
run_ids = [
    "run_20251127-162910_4t_vs_background_positional_job0",
    "run_20251127-162910_4t_vs_background_positional_job1",
    "run_20251127-162910_4t_vs_background_positional_job2",
    "run_20251127-162910_4t_vs_background_positional_job3",
]

# Import additional tools for manual loading
from thesis_ml.architectures.transformer_classifier.base import build_from_config as build_classifier  # noqa: E402
from thesis_ml.training_loops.transformer_classifier import _gather_meta  # noqa: E402

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading models...")
for run_id in run_ids:
    try:
        # Direct path construction
        run_dir = models_root / run_id
        cfg_path = run_dir / ".hydra" / "config.yaml"
        
        if not cfg_path.exists():
            print(f"✗ Config not found: {cfg_path}")
            continue
        
        # Load config
        cfg = OmegaConf.load(str(cfg_path))
        
        # Override data path
        cfg.data.path = str(data_path)
        
        # Populate meta if missing
        if not hasattr(cfg, "meta") or cfg.meta is None:
            train_dl_temp, val_dl_temp, test_dl_temp, meta = make_classification_dataloaders(cfg)
            _gather_meta(cfg, meta)
        
        # Build model
        model = build_classifier(cfg, cfg.meta).to(device)
        
        # Load weights
        best_val_path = run_dir / "best_val.pt"
        model_pt_path = run_dir / "model.pt"
        
        if best_val_path.exists():
            weights_path = best_val_path
        elif model_pt_path.exists():
            weights_path = model_pt_path
        else:
            print(f"✗ No weights found in {run_dir}")
            continue
        
        checkpoint = torch.load(str(weights_path), map_location=device, weights_only=False)
        state_dict = checkpoint["model_state_dict"] if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint else checkpoint
        model.load_state_dict(state_dict)
        model.eval()
        
        pos_enc_type = cfg.classifier.model.get("positional", "unknown")
        models_info[pos_enc_type] = {
            "run_id": run_id,
            "cfg": cfg,
            "model": model,
            "device": device,
        }
        
        print(f"✓ Loaded {run_id}")
        print(f"  Positional encoding: {pos_enc_type}")
        print(f"  Model dim: {cfg.classifier.model.dim}")
        print(f"  Layers: {cfg.classifier.model.depth}")
        print(f"  Heads: {cfg.classifier.model.heads}")
        print()
        
    except Exception as e:
        print(f"✗ Failed to load {run_id}: {e}")
        import traceback
        traceback.print_exc()
        print()

print(f"\nSuccessfully loaded {len(models_info)} models:")
print(f"Positional encoding types: {list(models_info.keys())}")

In [ ]:
# Load dataset (use config from first model)
sample_cfg = next(iter(models_info.values()))["cfg"]

# Override data path to use our specified location
sample_cfg.data.path = str(data_path)

print("Loading dataset...")
train_dl, val_dl, test_dl, meta = make_classification_dataloaders(sample_cfg)

print("✓ Dataset loaded")
print(f"  Path: {data_path}")
print(f"  n_tokens: {meta['n_tokens']}")
print(f"  n_classes: {meta['n_classes']}")
print(f"  has_globals: {meta['has_globals']}")
print(f"  Train batches: {len(train_dl)}")
print(f"  Val batches: {len(val_dl)}")
print(f"  Test batches: {len(test_dl)}")

## 2. Visualize Raw Positional Encoding Patterns

Extract and visualize the positional encoding matrices before they're added to embeddings.

**Note:** Rotary encoding doesn't have an additive positional encoding matrix - it applies rotations inside the attention mechanism.

In [ ]:
# Extract positional encoding matrices
pe_matrices = {}

for pe_type, info in models_info.items():
    model = info["model"]
    
    if pe_type == "none":
        # No positional encoding
        pe_matrices[pe_type] = None
        print(f"{pe_type}: No positional encoding (identity)")
        
    elif pe_type == "rotary":
        # Rotary is applied in attention, not additive
        # Extract the cached sin/cos patterns after they're computed
        rotary_emb = model.encoder.rotary_emb
        if rotary_emb is not None:
            print(f"{pe_type}: Applied in attention mechanism (not additive)")
            print(f"  head_dim: {rotary_emb.head_dim}")
            print(f"  base: {rotary_emb.base}")
            # We'll visualize this differently later
            pe_matrices[pe_type] = None
        else:
            print(f"{pe_type}: No rotary embedding found")
            pe_matrices[pe_type] = None
            
    else:
        # Sinusoidal or Learned
        pos_enc = model.pos_enc
        if pos_enc is not None and hasattr(pos_enc, 'pe'):
            pe_matrix = pos_enc.pe.detach().cpu().numpy()
            pe_matrices[pe_type] = pe_matrix
            print(f"{pe_type}: Shape {pe_matrix.shape} (tokens × model_dim)")
        else:
            pe_matrices[pe_type] = None
            print(f"{pe_type}: No positional encoding matrix found")

print("\n✓ Extracted positional encoding patterns")

In [ ]:
# Visualize positional encoding matrices as heatmaps
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_types = ['sinusoidal', 'learned']
for idx, pe_type in enumerate(plot_types):
    ax = axes[idx]
    
    if pe_type in pe_matrices and pe_matrices[pe_type] is not None:
        pe_matrix = pe_matrices[pe_type]
        
        # Plot heatmap
        im = ax.imshow(pe_matrix, aspect='auto', cmap='RdBu_r', 
                       vmin=-np.abs(pe_matrix).max(), vmax=np.abs(pe_matrix).max())
        ax.set_xlabel('Model Dimension', fontsize=12)
        ax.set_ylabel('Token Position', fontsize=12)
        ax.set_title(f'{pe_type.capitalize()} Positional Encoding\n({pe_matrix.shape[0]} tokens × {pe_matrix.shape[1]} dims)', 
                     fontsize=14, fontweight='bold')
        plt.colorbar(im, ax=ax, label='Encoding Value')
    else:
        ax.text(0.5, 0.5, f'{pe_type.capitalize()}\nNot available', 
                ha='center', va='center', fontsize=14, transform=ax.transAxes)
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

print("Heatmap shows how each position is encoded across all model dimensions")

In [ ]:
# Visualize per-token patterns: show how each token's encoding varies across dimensions
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

plot_types = ['sinusoidal', 'learned']
for idx, pe_type in enumerate(plot_types):
    ax = axes[idx]
    
    if pe_type in pe_matrices and pe_matrices[pe_type] is not None:
        pe_matrix = pe_matrices[pe_type]
        
        # Plot first 5 tokens to show pattern
        tokens_to_plot = min(5, pe_matrix.shape[0])
        for token_idx in range(tokens_to_plot):
            ax.plot(pe_matrix[token_idx, :], label=f'Token {token_idx}', alpha=0.7)
        
        ax.set_xlabel('Model Dimension', fontsize=12)
        ax.set_ylabel('Encoding Value', fontsize=12)
        ax.set_title(f'{pe_type.capitalize()} - Encoding Pattern Across Dimensions', 
                     fontsize=14, fontweight='bold')
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, f'{pe_type.capitalize()}\nNot available', 
                ha='center', va='center', fontsize=14, transform=ax.transAxes)
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Sinusoidal: Smooth wave patterns with different frequencies per dimension")
print("- Learned: Random-initialized patterns that were optimized during training")

In [ ]:
# For Rotary: visualize the sin/cos patterns
# Need to trigger a forward pass first to populate the cache

print("Visualizing Rotary (RoPE) sin/cos patterns...")
if 'rotary' in models_info:
    model = models_info['rotary']['model']
    rotary_emb = model.encoder.rotary_emb
    
    if rotary_emb is not None:
        # Get a sample batch to trigger forward pass
        sample_batch = next(iter(val_dl))
        device = models_info['rotary']['device']
        
        with torch.no_grad():
            # Forward pass to populate rotary cache
            if len(sample_batch) == 4:  # (tokens_cont, tokens_id, labels, mask)
                tokens_cont, tokens_id, labels, mask = sample_batch
                tokens_cont = tokens_cont.to(device)
                tokens_id = tokens_id.to(device)
                mask = mask.to(device)
                _ = model(tokens_cont, tokens_id, mask=mask)
            elif len(sample_batch) == 5:  # (tokens_cont, tokens_id, globals, mask, labels)
                tokens_cont, tokens_id, globals, mask, labels = sample_batch
                tokens_cont = tokens_cont.to(device)
                tokens_id = tokens_id.to(device)
                mask = mask.to(device)
                _ = model(tokens_cont, tokens_id, mask=mask)
            else:  # binned format
                tokens, labels, mask = sample_batch
                tokens = tokens.to(device)
                mask = mask.to(device)
                _ = model(tokens, mask=mask)
        
        # Extract cached sin/cos
        if hasattr(rotary_emb, '_cos_cached') and rotary_emb._cos_cached is not None:
            cos_cache = rotary_emb._cos_cached.squeeze().detach().cpu().numpy()  # [seq_len, head_dim]
            sin_cache = rotary_emb._sin_cached.squeeze().detach().cpu().numpy()
            
            fig, axes = plt.subplots(1, 2, figsize=(16, 6))
            
            # Plot cos
            im0 = axes[0].imshow(cos_cache, aspect='auto', cmap='RdBu_r',
                               vmin=-1, vmax=1)
            axes[0].set_xlabel('Head Dimension', fontsize=12)
            axes[0].set_ylabel('Sequence Position', fontsize=12)
            axes[0].set_title(f'Rotary Cosine Pattern\n({cos_cache.shape[0]} positions × {cos_cache.shape[1]} head_dims)', 
                            fontsize=14, fontweight='bold')
            plt.colorbar(im0, ax=axes[0], label='Cosine Value')
            
            # Plot sin
            im1 = axes[1].imshow(sin_cache, aspect='auto', cmap='RdBu_r',
                               vmin=-1, vmax=1)
            axes[1].set_xlabel('Head Dimension', fontsize=12)
            axes[1].set_ylabel('Sequence Position', fontsize=12)
            axes[1].set_title(f'Rotary Sine Pattern\n({sin_cache.shape[0]} positions × {sin_cache.shape[1]} head_dims)', 
                            fontsize=14, fontweight='bold')
            plt.colorbar(im1, ax=axes[1], label='Sine Value')
            
            plt.tight_layout()
            plt.show()
            
            print("✓ Rotary patterns visualized")
            print("  These are applied to Q and K in attention, not added to embeddings")
        else:
            print("  No rotary cache populated (cache may not exist or forward pass didn't trigger it)")
else:
    print("  No rotary model loaded")

## 3. Extract Layer-by-Layer Representations

Use forward hooks to capture intermediate representations at each stage:
1. After embedding (before positional encoding)
2. After positional encoding is added
3. After each transformer encoder block

In [ ]:
# Hook storage
class ActivationCapture:
    """Capture intermediate activations during forward pass."""
    
    def __init__(self):
        self.activations = {}
        self.hooks = []
    
    def register_hook(self, module, name):
        """Register a forward hook on a module."""
        def hook(module, input, output):
            # Handle different output types
            if isinstance(output, tuple):
                # Some modules return (output, attention_weights)
                self.activations[name] = output[0].detach().cpu()
            else:
                self.activations[name] = output.detach().cpu()
        
        handle = module.register_forward_hook(hook)
        self.hooks.append(handle)
    
    def clear(self):
        """Clear stored activations."""
        self.activations = {}
    
    def remove_hooks(self):
        """Remove all registered hooks."""
        for handle in self.hooks:
            handle.remove()
        self.hooks = []

print("✓ ActivationCapture class defined")

In [ ]:
# Function to extract representations from all models
def extract_layer_representations(models_info, data_loader, num_batches=1):
    """Extract layer-by-layer representations from all models.
    
    Parameters
    ----------
    models_info : dict
        Dictionary mapping PE type to model info
    data_loader : DataLoader
        DataLoader to get samples from
    num_batches : int
        Number of batches to process
    
    Returns
    -------
    dict
        Nested dict: {pe_type: {layer_name: tensor [B, T, D]}}
    """
    all_representations = {}
    all_labels = {}
    
    for pe_type, info in models_info.items():
        model = info["model"]
        device = info["device"]
        model.eval()
        
        # Create activation capture
        capture = ActivationCapture()
        
        # Register hooks on key modules
        capture.register_hook(model.embedding, "after_embedding")
        
        # Only register pos_enc hook if it exists and is not rotary
        if model.pos_enc is not None and pe_type not in ["none", "rotary"]:
            capture.register_hook(model.pos_enc, "after_pos_enc")
        
        # Register hooks on each encoder block
        for block_idx, block in enumerate(model.encoder.blocks):
            capture.register_hook(block, f"after_block_{block_idx}")
        
        # Initialize batch_activations with all possible keys
        possible_keys = ["after_embedding"]
        if model.pos_enc is not None and pe_type not in ["none", "rotary"]:
            possible_keys.append("after_pos_enc")
        possible_keys.extend([f"after_block_{i}" for i in range(len(model.encoder.blocks))])
        
        batch_activations = {key: [] for key in possible_keys}
        batch_labels = []
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(data_loader):
                if batch_idx >= num_batches:
                    break
                
                # Unpack batch - handle different formats
                if len(batch) == 4:  # Raw format: (tokens_cont, tokens_id, labels, mask)
                    tokens_cont, tokens_id, labels, mask = batch
                    tokens_cont = tokens_cont.to(device)
                    tokens_id = tokens_id.to(device)
                    mask = mask.to(device)
                    _ = model(tokens_cont, tokens_id, mask=mask)
                elif len(batch) == 5:  # Raw format with globals: (tokens_cont, tokens_id, globals, mask, labels)
                    tokens_cont, tokens_id, globals, mask, labels = batch
                    tokens_cont = tokens_cont.to(device)
                    tokens_id = tokens_id.to(device)
                    mask = mask.to(device)
                    _ = model(tokens_cont, tokens_id, mask=mask)
                else:  # Binned format: (tokens, labels, mask) or (tokens, globals, mask, labels)
                    if len(batch) == 3:
                        tokens, labels, mask = batch
                        tokens = tokens.to(device)
                        mask = mask.to(device)
                        _ = model(tokens, mask=mask)
                    else:
                        tokens, globals, mask, labels = batch
                        tokens = tokens.to(device)
                        mask = mask.to(device)
                        _ = model(tokens, mask=mask)
                
                # Store activations
                for key in capture.activations:
                    if key in batch_activations:
                        batch_activations[key].append(capture.activations[key])
                
                batch_labels.append(labels.cpu())
                capture.clear()
        
        # Concatenate batches
        representations = {}
        for key, act_list in batch_activations.items():
            if len(act_list) > 0:
                representations[key] = torch.cat(act_list, dim=0)
        
        all_representations[pe_type] = representations
        all_labels[pe_type] = torch.cat(batch_labels, dim=0)
        
        # Clean up hooks
        capture.remove_hooks()
        
        print(f"✓ Extracted representations for {pe_type}")
        print(f"  Layers captured: {list(representations.keys())}")
    
    return all_representations, all_labels

print("✓ Extraction function defined")

In [ ]:
# Extract representations from validation set
print("Extracting layer-by-layer representations...")
print("This may take a minute...\n")

representations, labels = extract_layer_representations(models_info, val_dl, num_batches=5)

print("\n✓ Extraction complete!")
print(f"\nModels analyzed: {list(representations.keys())}")
print(f"Sample counts per model: {labels[list(labels.keys())[0]].shape[0]}")

# Display shape information
sample_pe_type = list(representations.keys())[0]
print(f"\nExample shapes for '{sample_pe_type}':")
for layer_name, tensor in representations[sample_pe_type].items():
    print(f"  {layer_name}: {tensor.shape} [batch, tokens, dim]")

## 4. Visualize Representations in Model Dimensions

Analyze how representations evolve through the network by looking at:
- Token-wise L2 norms (representation magnitude)
- Pairwise token similarities (cosine similarity)
- How these differ across positional encoding types

In [ ]:
# Compute token-wise L2 norms across layers
def compute_l2_norms(representations):
    """Compute L2 norm for each token across all layers."""
    norms = {}
    
    for pe_type, layers in representations.items():
        norms[pe_type] = {}
        for layer_name, tensor in layers.items():
            # tensor shape: [B, T, D]
            # Compute L2 norm along dimension axis
            l2 = torch.norm(tensor, p=2, dim=-1)  # [B, T]
            norms[pe_type][layer_name] = l2.mean(dim=0).numpy()  # Average over batch -> [T]
    
    return norms

# Compute norms
print("Computing L2 norms...")
l2_norms = compute_l2_norms(representations)
print("✓ L2 norms computed")

In [ ]:
# Plot L2 norms evolution through layers
# Select key layers to visualize
sample_pe = list(l2_norms.keys())[0]
num_blocks = len([k for k in l2_norms[sample_pe] if 'block' in k])

layers_to_plot = ['after_embedding']
if 'after_pos_enc' in l2_norms[sample_pe]:
    layers_to_plot.append('after_pos_enc')
# Add first, middle, and last block
layers_to_plot.extend(['after_block_0', f'after_block_{num_blocks//2}', f'after_block_{num_blocks-1}'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

pe_types_ordered = ['none', 'sinusoidal', 'learned', 'rotary']
pe_types_available = [pt for pt in pe_types_ordered if pt in l2_norms]

for idx, pe_type in enumerate(pe_types_available):
    ax = axes[idx]
    
    for layer_name in layers_to_plot:
        if layer_name in l2_norms[pe_type]:
            norms = l2_norms[pe_type][layer_name]
            ax.plot(norms, label=layer_name, marker='o', alpha=0.7)
    
    ax.set_xlabel('Token Position', fontsize=12)
    ax.set_ylabel('L2 Norm', fontsize=12)
    ax.set_title(f'{pe_type.capitalize()} - Token Representation Magnitude', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(len(pe_types_available), 4):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("Observations:")
print("- L2 norms show how representation magnitude changes through layers")
print("- Different PE types may show different patterns of token differentiation")

In [ ]:
# Compute pairwise token similarity matrices
def compute_similarity_matrix(tensor):
    """Compute cosine similarity between all token pairs.
    
    Parameters
    ----------
    tensor : torch.Tensor
        Shape [B, T, D]
    
    Returns
    -------
    np.ndarray
        Similarity matrix [T, T] averaged over batch
    """
    # Average over batch
    token_vecs = tensor.mean(dim=0)  # [T, D]
    
    # Compute cosine similarity
    from torch.nn.functional import cosine_similarity
    
    T = token_vecs.shape[0]
    sim_matrix = np.zeros((T, T))
    
    for i in range(T):
        for j in range(T):
            if i == j:
                sim_matrix[i, j] = 1.0
            else:
                sim = cosine_similarity(
                    token_vecs[i:i+1], 
                    token_vecs[j:j+1], 
                    dim=1
                ).item()
                sim_matrix[i, j] = sim
    
    return sim_matrix

print("✓ Similarity computation function defined")

In [ ]:
# Visualize similarity matrices at key layers
# Compare after positional encoding is added

# Ensure pe_types_available is defined
pe_types_ordered = ['none', 'sinusoidal', 'learned', 'rotary']
pe_types_available = [pt for pt in pe_types_ordered if pt in representations]

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

layer_to_compare = 'after_pos_enc'  # or 'after_embedding' if no pos_enc

for idx, pe_type in enumerate(pe_types_available):
    ax = axes[idx]
    
    # Find appropriate layer
    if layer_to_compare in representations[pe_type]:
        layer_tensor = representations[pe_type][layer_to_compare]
        actual_layer = layer_to_compare
    elif 'after_embedding' in representations[pe_type]:
        layer_tensor = representations[pe_type]['after_embedding']
        actual_layer = 'after_embedding'
    else:
        ax.text(0.5, 0.5, f'{pe_type}\nNo data', 
                ha='center', va='center', transform=ax.transAxes)
        continue
    
    # Compute similarity matrix
    sim_matrix = compute_similarity_matrix(layer_tensor)
    
    # Plot
    im = ax.imshow(sim_matrix, cmap='RdYlGn', vmin=-1, vmax=1, aspect='auto')
    ax.set_xlabel('Token Index', fontsize=12)
    ax.set_ylabel('Token Index', fontsize=12)
    ax.set_title(f'{pe_type.capitalize()} - Token Similarity\n{actual_layer}', 
                 fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Cosine Similarity')

# Hide unused subplots
for idx in range(len(pe_types_available), 4):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- Diagonal is always 1.0 (token similarity with itself)")
print("- Off-diagonal patterns show how PE creates token differentiation")
print("- 'None' should show more uniform similarity (less position info)")
print("- Sinusoidal/Learned/Rotary should show position-dependent patterns")

## 5. Dimensionality Reduction: PCA and t-SNE

Project high-dimensional representations to 2D/3D to visualize:
- Class separability (4t vs background)
- How separation emerges through layers
- Differences between positional encoding strategies

In [ ]:
# Import dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

print("✓ Dimensionality reduction imports loaded")

In [ ]:
# Prepare data for dimensionality reduction
def prepare_for_reduction(tensor, pooling='mean'):
    """
    Prepare tensor for dimensionality reduction.
    
    Parameters
    ----------
    tensor : torch.Tensor
        Shape [B, T, D]
    pooling : str
        'mean' or 'cls' (use first token)
    
    Returns
    -------
    np.ndarray
        Shape [B, D]
    """
    if pooling == 'mean':
        # Average over tokens
        pooled = tensor.mean(dim=1)  # [B, D]
    elif pooling == 'cls':
        # Use first token (CLS)
        pooled = tensor[:, 0, :]  # [B, D]
    else:
        raise ValueError(f"Unknown pooling: {pooling}")
    
    return pooled.numpy()

print("✓ Data preparation function defined")

In [ ]:
# PCA visualization - compare final layer across all PE types

# Ensure pe_types_available and num_blocks are defined
pe_types_ordered = ['none', 'sinusoidal', 'learned', 'rotary']
pe_types_available = [pt for pt in pe_types_ordered if pt in representations]

# Determine num_blocks from first available model
if pe_types_available:
    sample_pe = pe_types_available[0]
    num_blocks = len([k for k in representations[sample_pe] if 'block' in k])
    final_layer = f'after_block_{num_blocks-1}'
else:
    final_layer = 'after_block_0'  # fallback

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, pe_type in enumerate(pe_types_available):
    ax = axes[idx]
    
    if final_layer in representations[pe_type]:
        # Get final layer representations
        tensor = representations[pe_type][final_layer]
        X = prepare_for_reduction(tensor, pooling='mean')
        y = labels[pe_type].numpy()
        
        # Apply PCA
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X)
        
        # Plot by class
        for class_idx in [0, 1]:
            mask = y == class_idx
            class_name = '4t (signal)' if class_idx == 1 else 'Background'
            ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                      label=class_name, alpha=0.6, s=20)
        
        ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)', fontsize=12)
        ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)', fontsize=12)
        ax.set_title(f'{pe_type.capitalize()} - PCA of Final Layer\nTotal variance: {pca.explained_variance_ratio_[:2].sum():.1%}', 
                     fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, f'{pe_type}\nNo data', 
                ha='center', va='center', transform=ax.transAxes)

# Hide unused subplots
for idx in range(len(pe_types_available), 4):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\nPCA shows linear separability of classes")
print("Better separation = easier classification task")

In [ ]:
# t-SNE visualization - focuses on local structure
print("Computing t-SNE (this may take a minute)...")

# Ensure pe_types_available and final_layer are defined
pe_types_ordered = ['none', 'sinusoidal', 'learned', 'rotary']
pe_types_available = [pt for pt in pe_types_ordered if pt in representations]

# Determine num_blocks and final_layer
if pe_types_available:
    sample_pe = pe_types_available[0]
    num_blocks = len([k for k in representations[sample_pe] if 'block' in k])
    final_layer = f'after_block_{num_blocks-1}'
else:
    final_layer = 'after_block_0'  # fallback

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, pe_type in enumerate(pe_types_available):
    print(f"  Processing {pe_type}...")
    ax = axes[idx]
    
    if final_layer in representations[pe_type]:
        # Get final layer representations
        tensor = representations[pe_type][final_layer]
        X = prepare_for_reduction(tensor, pooling='mean')
        y = labels[pe_type].numpy()
        
        # Apply t-SNE
        tsne = TSNE(n_components=2, perplexity=30, random_state=42, max_iter=1000)
        X_tsne = tsne.fit_transform(X)
        
        # Plot by class
        for class_idx in [0, 1]:
            mask = y == class_idx
            class_name = '4t (signal)' if class_idx == 1 else 'Background'
            ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                      label=class_name, alpha=0.6, s=20)
        
        ax.set_xlabel('t-SNE 1', fontsize=12)
        ax.set_ylabel('t-SNE 2', fontsize=12)
        ax.set_title(f'{pe_type.capitalize()} - t-SNE of Final Layer', 
                     fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, f'{pe_type}\nNo data', 
                ha='center', va='center', transform=ax.transAxes)

# Hide unused subplots
for idx in range(len(pe_types_available), 4):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\n✓ t-SNE complete")
print("t-SNE reveals local clustering structure")
print("Well-separated clusters indicate better learned representations")

In [ ]:
# Evolution through layers - show how PCA separation improves
# Pick one PE type to show layer evolution

# Ensure pe_types_available and num_blocks are defined
pe_types_ordered = ['none', 'sinusoidal', 'learned', 'rotary']
pe_types_available = [pt for pt in pe_types_ordered if pt in representations]

if not pe_types_available:
    print("No models available for layer evolution visualization")
else:
    pe_for_evolution = 'sinusoidal' if 'sinusoidal' in representations else pe_types_available[0]
    
    # Determine num_blocks
    num_blocks = len([k for k in representations[pe_for_evolution] if 'block' in k])
    
    # Select layers to show evolution
    evolution_layers = ['after_embedding']
    if 'after_pos_enc' in representations[pe_for_evolution]:
        evolution_layers.append('after_pos_enc')
    evolution_layers.extend([f'after_block_{i}' for i in [0, num_blocks//2, num_blocks-1]])
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, layer_name in enumerate(evolution_layers[:6]):  # Max 6 subplots
        ax = axes[idx]
        
        if layer_name in representations[pe_for_evolution]:
            tensor = representations[pe_for_evolution][layer_name]
            X = prepare_for_reduction(tensor, pooling='mean')
            y = labels[pe_for_evolution].numpy()
            
            # Apply PCA
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)
            
            # Plot by class
            for class_idx in [0, 1]:
                mask = y == class_idx
                class_name = '4t' if class_idx == 1 else 'Bkg'
                ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                          label=class_name, alpha=0.6, s=15)
            
            var_explained = pca.explained_variance_ratio_[:2].sum()
            ax.set_xlabel('PC1', fontsize=10)
            ax.set_ylabel('PC2', fontsize=10)
            ax.set_title(f'{layer_name}\nVar: {var_explained:.1%}', fontsize=12, fontweight='bold')
            ax.legend(fontsize=9)
            ax.grid(True, alpha=0.3)
        else:
            ax.axis('off')
    
    plt.suptitle(f'Layer Evolution - {pe_for_evolution.capitalize()} PE', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"\nShowing how class separation evolves through layers for {pe_for_evolution}")
    print("Watch how the two classes become more separable in deeper layers")

## 6. Performance Comparison

Run inference on all models and compare their classification performance to quantify why positional encoding matters.

In [ ]:
# Run inference on all models
from thesis_ml.reports.inference.classification import run_classification_inference

print("Running inference on all models...")
print("This will compute predictions and metrics on the test set.\n")

# Prepare models list
models_list = []
for _pe_type, info in models_info.items():
    models_list.append((info['run_id'], info['cfg'], info['model']))

# Run inference
try:
    inference_results = run_classification_inference(
        models=models_list,
        dataset_cfg=sample_cfg,
        split="test",
        inference_cfg={
            "batch_size": 512,
            "autocast": False,
            "seed": 42,
            "max_samples": None,
        }
    )
    
    print("\n✓ Inference complete!")
    print(f"Models evaluated: {list(inference_results.keys())}")
except Exception as e:
    print(f"\n✗ Inference failed: {e}")
    import traceback
    traceback.print_exc()
    inference_results = {}

In [ ]:
# Create performance metrics table
import pandas as pd

if inference_results:
    metrics_data = []
    
    for run_id, metrics in inference_results.items():
        # Find corresponding PE type
        pe_type = None
        for pt, info in models_info.items():
            if info['run_id'] == run_id:
                pe_type = pt
                break
        
        if pe_type and isinstance(metrics, dict):
            metrics_data.append({
                'Positional Encoding': pe_type.capitalize(),
                'Accuracy': f"{metrics.get('accuracy', 0.0):.4f}",
                'Precision': f"{metrics.get('precision', 0.0):.4f}",
                'Recall': f"{metrics.get('recall', 0.0):.4f}",
                'F1': f"{metrics.get('f1', 0.0):.4f}",
                'AUROC': f"{metrics.get('auroc', 0.0):.4f}",
            })
    
    if metrics_data:
        metrics_df = pd.DataFrame(metrics_data)
        metrics_df = metrics_df.sort_values('AUROC', ascending=False)
        
        print("\n" + "="*80)
        print("PERFORMANCE COMPARISON - 4t vs Background Classification")
        print("="*80)
        print(metrics_df.to_string(index=False))
        print("="*80)
        
        # Highlight best performance
        best_pe = metrics_df.iloc[0]['Positional Encoding']
        best_auroc = metrics_df.iloc[0]['AUROC']
        print(f"\n🏆 Best performing: {best_pe} (AUROC: {best_auroc})")
    else:
        print("No metrics data available")
else:
    print("No inference results available. Please run inference first.")

In [ ]:
# Plot ROC curves for all models
if inference_results:
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    has_data = False
    for run_id, metrics in inference_results.items():
        # Find corresponding PE type
        pe_type = None
        for pt, info in models_info.items():
            if info['run_id'] == run_id:
                pe_type = pt
                break
        
        if pe_type and isinstance(metrics, dict) and 'roc_curves' in metrics:
            roc_curves = metrics['roc_curves']
            # Try class 1 (4t signal), fallback to first available class
            class_key = 1 if 1 in roc_curves else (list(roc_curves.keys())[0] if roc_curves else None)
            
            if class_key is not None:
                roc_data = roc_curves[class_key]
                if isinstance(roc_data, dict) and 'fpr' in roc_data and 'tpr' in roc_data:
                    fpr = roc_data['fpr']
                    tpr = roc_data['tpr']
                    auroc = metrics.get('auroc', 0.0)
                    
                    ax.plot(fpr, tpr, label=f'{pe_type.capitalize()} (AUROC={auroc:.4f})', 
                            linewidth=2.5, alpha=0.8)
                    has_data = True
    
    if has_data:
        # Plot diagonal
        ax.plot([0, 1], [0, 1], 'k--', linewidth=1.5, alpha=0.5, label='Random')
        
        ax.set_xlabel('False Positive Rate', fontsize=14)
        ax.set_ylabel('True Positive Rate', fontsize=14)
        ax.set_title('ROC Curves - Positional Encoding Comparison\n4t vs Background Classification', 
                     fontsize=16, fontweight='bold')
        ax.legend(loc='lower right', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])
        
        plt.tight_layout()
        plt.show()
        
        print("\nROC curves show the trade-off between true positive rate and false positive rate")
        print("Higher curve = better classification performance")
    else:
        print("No ROC curve data available")
else:
    print("No inference results available. Please run inference first.")

In [ ]:
# Visualize prediction score distributions
if inference_results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    pe_types_with_scores = []
    for run_id, metrics in inference_results.items():
        for pt, info in models_info.items():
            if info['run_id'] == run_id:
                pe_types_with_scores.append((pt, metrics))
                break
    
    for idx, (pe_type, metrics) in enumerate(pe_types_with_scores[:4]):
        ax = axes[idx]
        
        if isinstance(metrics, dict) and 'per_event_scores' in metrics and 'per_event_labels' in metrics:
            scores = metrics['per_event_scores']
            true_labels = metrics['per_event_labels']
            
            # Ensure scores and labels are numpy arrays
            if isinstance(scores, torch.Tensor):
                scores = scores.numpy()
            if isinstance(true_labels, torch.Tensor):
                true_labels = true_labels.numpy()
            
            # Ensure they are 1D arrays (flatten if needed)
            scores = np.asarray(scores).flatten()
            true_labels = np.asarray(true_labels).flatten()
            
            # Plot distributions for each class
            for class_idx in [0, 1]:
                class_mask = true_labels == class_idx
                # Check if mask is array-like and has any True values
                if hasattr(class_mask, 'sum') and class_mask.sum() > 0:
                    class_scores = scores[class_mask]
                    class_name = '4t (signal)' if class_idx == 1 else 'Background'
                    ax.hist(class_scores, bins=50, alpha=0.6, label=class_name, density=True)
            
            ax.set_xlabel('Predicted Signal Probability', fontsize=12)
            ax.set_ylabel('Density', fontsize=12)
            ax.set_title(f'{pe_type.capitalize()} - Score Distribution', 
                         fontsize=14, fontweight='bold')
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.axvline(0.5, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='Threshold=0.5')
        else:
            ax.text(0.5, 0.5, f'{pe_type}\nNo score data', 
                    ha='center', va='center', transform=ax.transAxes)
    
    # Hide unused subplots
    for idx in range(len(pe_types_with_scores), 4):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\nScore distributions show how well the model separates the two classes")
    print("Good separation = distinct peaks for signal and background")
else:
    print("No inference results available. Please run inference first.")

## 7. Summary and Conclusions

### Key Findings

This notebook demonstrates how different positional encoding strategies affect transformer-based classification:

#### **Positional Encoding Patterns**
- **Sinusoidal**: Fixed wave patterns with varying frequencies across dimensions
- **Learned**: Trainable embeddings optimized during training
- **Rotary (RoPE)**: Applied in attention mechanism (not additive to embeddings)
- **None**: No positional information (permutation invariant)

#### **Representation Evolution**
- Positional encodings create distinct token relationships early in the network
- Token similarity patterns differ significantly between PE types
- Class separation emerges and strengthens through transformer layers

#### **Performance Impact**
The metrics table above shows quantitative differences in classification performance. Key observations:
- Models with positional encoding generally outperform "none"
- Different PE strategies (sinusoidal vs learned vs rotary) show varying effectiveness
- PCA/t-SNE visualizations correlate with classification performance

#### **Why Positional Encoding Helps**
1. **Token Differentiation**: Adds unique information to each token position
2. **Sequence Structure**: Allows the model to learn position-dependent patterns
3. **Better Representations**: Creates more separable feature spaces (shown in PCA/t-SNE)
4. **Improved Classification**: Translates to higher AUROC and accuracy

### Particle Physics Context
For particle physics events, positional encoding can capture:
- Particle ordering (e.g., by pT, η, φ)
- Detector geometry information
- Spatial relationships between particles

However, true permutation invariance (PE=none) may be preferred when particle order is arbitrary.